# Question-Generation RAG 


This experiment investigates how question-driven chunk indexing impacts the RAG pipeline’s
performance in both relevant retrieval and answer accuracy. The approach differs from
conventional fixed-length chunking or the previous semantic chunking, in terms of generating
relevant questions based on each document and linking them to the corresponding chunks.
Alongside the text chunks, also the respective questions are embedded when establishing the
retriever, making similarity search rather based on questions not the underlying text chunks.
The direct mapping between question and respective chunks aims to optimise retrieval. The
assumption is that this approach will serve user queries better, by structuring the information
in the knowledge base in a way users would naturally need and access. It basically simulates a
realistic QA process making it highly efficient for situations where questions are repetitive, but
also for all other use-cases.

In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Question_Generation"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

In [3]:
import langchain
import tiktoken
import sys
import os
import re
from langchain_community.document_loaders import PyPDFLoader
from pydantic import BaseModel, Field
from langchain.docstore.document import Document
from enum import Enum
from typing import Any, Dict, List, Tuple
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from typing import List
from tqdm import tqdm
import pandas as pd
from IPython.display import display

In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens = 4000
)

In [6]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

## Question generation (Code) - Generated in another file

In [ ]:
class QuestionGeneration(Enum):
    """
    Enum class to specify the level of question generation for document processing.

    Attributes:
        DOCUMENT_LEVEL (int): Represents question generation at the entire document level.
        FRAGMENT_LEVEL (int): Represents question generation at the individual text fragment level.
    """
    DOCUMENT_LEVEL = 30
    FRAGMENT_LEVEL = 5

In [ ]:
#Depending on the model, for Mitral 7B it can be max 8000, for Llama 3.1 8B 128k
DOCUMENT_MAX_TOKENS = 4000
DOCUMENT_OVERLAP_TOKENS = 100

#Embeddings and text similarity calculated on shorter texts
FRAGMENT_MAX_TOKENS = 400
FRAGMENT_OVERLAP_TOKENS = 50

#Questions generated on document or fragment level
QUESTION_GENERATION = [QuestionGeneration.DOCUMENT_LEVEL, QuestionGeneration.FRAGMENT_LEVEL]

#how many questions will be generated for specific document or fragment
# QUESTIONS_PER_DOCUMENT = 2

QUESTIONS_PER_DOCUMENT_LEVEL = 30
QUESTIONS_PER_FRAGMENT_LEVEL = 5


In [ ]:
class QuestionList(BaseModel):
    question_list: List[str] = Field(..., title="List of questions generated for the document or fragment")

In [ ]:
def clean_and_filter_questions(questions: List[str]) -> List[str]:
    """
    Cleans and filters a list of questions.

    Args:
        questions (List[str]): A list of questions to be cleaned and filtered.

    Returns:
        List[str]: A list of cleaned and filtered questions that end with a question mark.
    """
    cleaned_questions = []
    for question in questions:
        cleaned_question = re.sub(r'^\d+\.\s*', '', question.strip())
        if cleaned_question.endswith('?'):
            cleaned_questions.append(cleaned_question)
    return cleaned_questions


In [ ]:
def generate_questions(texts: List[str], num_questions: int) -> List[List[str]]:
    """
    Generates questions for multiple text inputs.

    Args:
        texts (List[str]): A list of text inputs.
        num_questions (int): Number of questions to generate per document or fragment.

    Returns:
        List[List[str]]: A list containing lists of generated questions for each text.
    """
    generated_questions = []
    
    for i, text in enumerate(tqdm(texts, total=len(texts), desc="Generating Questions", unit="doc")):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        template = "Using the context data: {context}\n\nGenerate a list of exactly {num_questions} " \
                   "possible questions that can be asked about this context. Ensure the questions are " \
                   "directly answerable within the context and do not include any answers or headers. " \
                   "Separate the questions with a new line character."

        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | llm.with_structured_output(QuestionList)

        input_data = {"context": text, "num_questions": num_questions}  # ✅ Fixed missing variable

        result = chain.invoke(input_data)
        questions = result.question_list
        filtered_questions = clean_and_filter_questions(questions)
        
        filtered_questions = filtered_questions[:num_questions]
        
        generated_questions.append(list(set(filtered_questions)))

    return generated_questions


In [ ]:
from typing import List
import re
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_document(document: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    """
    Splits a document into smaller chunks of text using RecursiveCharacterTextSplitter.

    Args:
        document (str): The text of the document to be split.
        chunk_size (int): The size of each chunk.
        chunk_overlap (int): The number of overlapping characters between consecutive chunks.

    Returns:
        List[str]: A list of text chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    # Split document into chunks
    split_docs = text_splitter.split_text(document)

#     # Create and print a table of text chunks
#     df = pd.DataFrame({
#         "Chunk Index": range(len(split_docs)),
#         "Text Chunk": split_docs
#     })

#     print("\n===== Top 3 Text Chunks =====\n")
#     print(df.head(3).to_string(index=False))  # Print only the first 3 rows
#     print("\n=============================\n")

    return split_docs


## The Process

In [ ]:
def process_documents_real(content: str) -> List[Document]:
    """
    A version of process_documents that processes *all* document chunks and fragments.

    Args:
        content (str): The content of the document to process.

    Returns:
        List[Document]: A list of processed documents ready for embedding.
    """
    # Remove slicing for max_docs
    text_documents = split_document(content, DOCUMENT_MAX_TOKENS, DOCUMENT_OVERLAP_TOKENS)
    print(f'✅ Processing only {len(text_documents)} DOCUMENT chunks (out of {len(text_documents)})')

    documents = []
    counter = 0

    for i, text_document in enumerate(tqdm(text_documents, desc="Processing Limited Documents", unit="doc")):
        # Store DOCUMENT-LEVEL CHUNK
        original_document = Document(
            page_content=text_document,
            metadata={"type": "ORIGINAL", "index": counter, "parent_doc": None}  # Mark as DOCUMENT chunk
        )
        documents.append(original_document)
        doc_index = counter  # Store index for linking questions
        counter += 1

        # ✅ Generate DOCUMENT-LEVEL Questions
        if QuestionGeneration.DOCUMENT_LEVEL in QUESTION_GENERATION:
            document_questions = generate_questions([text_document], QUESTIONS_PER_DOCUMENT_LEVEL)
            document_questions = document_questions[0] if document_questions else []

            for idx, question in enumerate(document_questions):
                q_doc = Document(
                    page_content=str(question),
                    metadata={"type": "AUGMENTED", "index": counter + idx, "parent_doc": doc_index}  # Link to DOCUMENT
                )
                documents.append(q_doc)
            counter += len(document_questions)

        # Remove slicing for max_fragments
        text_fragments = split_document(text_document, FRAGMENT_MAX_TOKENS, FRAGMENT_OVERLAP_TOKENS)

        for j, text_fragment in enumerate(tqdm(text_fragments, desc=f"Splitting Doc {i}", leave=False, unit="fragment")):
            # Store FRAGMENT-LEVEL CHUNK
            original_fragment = Document(
                page_content=text_fragment,
                metadata={"type": "ORIGINAL", "index": counter, "parent_doc": doc_index}  # Link to parent DOCUMENT
            )
            documents.append(original_fragment)
            fragment_index = counter
            counter += 1

            # ✅ Generate FRAGMENT-LEVEL Questions
            if QuestionGeneration.FRAGMENT_LEVEL in QUESTION_GENERATION:
                fragment_questions = generate_questions([text_fragment], QUESTIONS_PER_FRAGMENT_LEVEL)
                fragment_questions = fragment_questions[0] if fragment_questions else []

                for idx, question in enumerate(fragment_questions):
                    q_doc = Document(
                        page_content=str(question),
                        metadata={"type": "AUGMENTED", "index": counter + idx, "parent_index": fragment_index}  # Link to FRAGMENT
                    )
                    documents.append(q_doc)
                counter += len(fragment_questions)

    print(f'✅ Test Run Complete: {len(documents)} total DOCUMENTS, FRAGMENTS, and QUESTIONS processed.')

    return documents


In [ ]:
process_documents_real(document)

In [ ]:
def create_vector_store(docs):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local("nvidia_large_questions_generated")
    return vectorstore.as_retriever()

# Create retriever with tracing
retriever = create_vector_store(my_docs)


# Now back to this code

## Loading Vector Store

In [7]:
# After you have loaded the FAISS store:
new_vector_store = FAISS.load_local(
    "nvidia_large_questions_generated", 
    embeddings,  
    allow_dangerous_deserialization=True
)

retriever = new_vector_store.as_retriever()

# Access the docstore
docstore = new_vector_store.docstore

# The docstore is typically an InMemoryDocstore with a ._dict that holds all the Documents
all_docs = list(docstore._dict.values())

print(f"Retrieved {len(all_docs)} documents from FAISS docstore.")
print("Sample metadata:", all_docs[0].metadata)
print("Sample content:", all_docs[0].page_content[:200], "...")


Retrieved 10949 documents from FAISS docstore.
Sample metadata: {'type': 'ORIGINAL', 'index': 0, 'parent_doc': None}
Sample content: Table of Contents
UNITED STATESSECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________________________________________________________________
FORM 10-K
☒ ANNUAL ...


## Creating the link to parents after retrieval

In [8]:
@traceable
def retrieve_correct_document_test(
    retrieved_docs_with_scores: List[tuple], 
    all_docs: List[Document]
) -> List[Document]:
    """
    Replaces retrieved 'AUGMENTED' documents with their corresponding parent document or fragment.
    
    If the retrieved doc has 'parent_index', it retrieves the fragment.
    If the retrieved doc has 'parent_doc', it retrieves the full document.

    Args:
        retrieved_docs_with_scores (List[tuple]): List of (document, similarity_score) tuples.
        all_docs (List[Document]): Full list of documents, including original and augmented.

    Returns:
        List[Document]: List where AUGMENTED docs are replaced with their parent document/fragment.
    """
    final_docs = []

    for doc, score in retrieved_docs_with_scores:
        if doc.metadata.get("type") == "AUGMENTED":
            # Check for parent references
            parent_index = doc.metadata.get("parent_index")
            parent_doc = doc.metadata.get("parent_doc")
            original_doc = None

            # First try to get the fragment (if parent_index is present)
            if parent_index is not None:
                original_doc = next(
                    (d for d in all_docs if d.metadata.get("index") == parent_index),
                    None
                )
            
            # If no fragment found, try to get the parent document
            if original_doc is None and parent_doc is not None:
                original_doc = next(
                    (d for d in all_docs if d.metadata.get("index") == parent_doc),
                    None
                )

            # Append the original doc if found, otherwise fallback to the question doc itself
            final_docs.append(original_doc if original_doc else doc)

        else:
            # If already an original doc, add it as is
            final_docs.append(doc)

    return final_docs


# RAG pipeline: The Chain

In [9]:
def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {
            "context": lambda x: "\n\n".join(x["retrieved_contexts"]),  # Use retrieved_contexts directly
            "input": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()

In [10]:
def display_retrieved_with_originals(documents):
    """
    Creates and displays a DataFrame with relevant information from retrieved documents.
    Gets also 

    Args:
        documents (List[Document]): A list of retrieved Document objects.
    """
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", f"Document {doc.metadata.get('index', 'Unknown')}"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Document Type": doc.metadata.get("type", "Unknown"),
            "Similarity Score": round(doc.metadata.get("similarity_score", 1.0), 4),  # Default to 1.0 if not available
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters for clarity
        }
        for doc in documents
    ])

    # Ensure full content is displayed in the DataFrame
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width for better visibility

    # Display the DataFrame
    display(df)

In [11]:
@traceable
def query_rag(question: str, docs: List[Document], top_k=4):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in a table.

    Args:
        question (str): The question to query.
        test_docs (List[Document]): The full set of stored documents.
        top_k (int, optional): Number of top documents to retrieve. Defaults to 4.

    Returns:
        dict: Contains the generated answer and retrieved contexts.
    """
    # Retrieve the top-k chunks with similarity scores
    retrieved_docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=top_k)

    # Extract document details into a structured DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters for clarity
        }
        for doc, score in retrieved_docs_with_scores
    ])

    # Ensure full content is displayed in the DataFrame
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width for better visibility

    # Display the DataFrame
    display(df)

    # Retrieve corrected documents based on parent relationships
    corrected_retrieved = retrieve_correct_document_test(retrieved_docs_with_scores, docs)

    # Display documents with original references
    display_retrieved_with_originals(corrected_retrieved)

    # Extract contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc in corrected_retrieved]

    # Run the RAG pipeline with the retrieved context
    answer = rag_chain.invoke({
        "input": question, 
        "retrieved_contexts": retrieved_contexts
    })

    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }




In [12]:
result = query_rag("When did the CEO join NVIDIA?", all_docs)

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4731,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
1,Unknown Document,N/A,0.4731,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
2,Unknown Document,N/A,0.4731,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
3,Unknown Document,N/A,0.4732,Who is the President and Chief Executive Officer of NVIDIA Corporation?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."


In [13]:
result["answer"]

'The provided context does not specify when Jen-Hsun Huang, the CEO of NVIDIA, joined the company. Therefore, I cannot provide an answer to the question based on the given information.'

In [15]:
result["contexts"][1]

'Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney'

# RAG Dataset

In [16]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [17]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_4845/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators Propositions

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [18]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"], all_docs)
    return {"answer": response["answer"]}

In [19]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"], all_docs)
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [20]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [21]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [26]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="Question_Generation",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Question_Generation-2294d7ac' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=2f0e02ee-f10f-4b99-a855-485baac2e7af




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4067,What markets does NVIDIA target with its full-stack innovation approach?...
1,Unknown Document,N/A,0.4946,What are the main applications of NVIDIA's computer graphics platforms?...
2,Unknown Document,N/A,0.5127,What industries are mentioned as benefiting from NVIDIA's technology?...
3,Unknown Document,N/A,0.5156,What market does the company serve in relation to NVIDIA GPUs?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 805,N/A,ORIGINAL,1.0,"hardware, a reference sensor set that supports full self-driving capability as well as an open, modular DRIVE software platform for autonomous driving, mapping,and parking services, and intelligent in-vehicle experiences.\nIn addition, we offer a scalable data center-based simulation solution, NVIDIA DRIVE Sim, based on NVIDIA Omniverse software, for digital cockpit\ndevelopment, as well as for testing and validating a self-driving platform. Our unique end-to-end, software-defined approach is designed for continuousinnovation and continuous development, enabling cars to receive over-the-air updates to add new features and capabilities throughout the life of a vehicle.\nBusiness Strategies\nNVIDIA’s key strategies that shape our overall business approach include:\nAdvancing the NVIDIA accelerated computing platform. Our accelerated computing platform can solve complex problems in significantly less time and with\nlower power consumption than alternative computational approaches. Indeed, it ca..."
1,Document 914,N/A,ORIGINAL,1.0,"entertainment and professional visualization applications and create new virtual world and simulation capabilities. Our technologies are instrumental in drivingthe gaming, design, and creative industries forward, as developers leverage our libraries and algorithms to deliver an optimized experience on our GeForce andNVIDIA RTX platforms. Our computer graphics platforms leverage AI end-to-end, from the developer tools and cloud services to the Tensor Cores included in all\nRTX-class GPUs. For example, NVIDIA Avatar Cloud Engine, or ACE, is a suite of technologies that help developers bring digital avatars to life with generativeAI, running in the cloud or locally on the PC. GeForce Experience enhances each gamer’s experience by optimizing their PC’s settings, as well as enabling the\nrecording and sharing of gameplay. Our Studio drivers enhance and accelerate a number of popular creative applications. Omniverse is real-time 3D designcollaboration and virtual world simulation software that..."
2,Document 206,N/A,ORIGINAL,1.0,"Forward-Looking Statements\nThis Annual Report on Form 10-K contains forward-looking statements which are based on our management's beliefs and assumptions and on informationcurrently available to our management. In some cases, you can identify forward-looking statements by terms such as “may,” “will,” “should,” “could,” “goal,”\n“would,” “expect,” “plan,” “anticipate,” “believe,” “estimate,” “project,” “predict,” “potential,” and similar expressions intended to identify forward-lookingstatements. These statements involve known and unknown risks, uncertainties and other factors, which may cause our actual results, performance, time framesor achievements to be materially different from any future results, performance, time frames or achievements expressed or implied by the forward-looking\nstatements. We discuss many of these risks, uncertainties, and other factors in this Annual Report on Form 10-K in greater detail under the heading “RiskFactors.” Given these risks, uncertainties, and ot..."
3,Document 727,N/A,ORIGINAL,1.0,"Professional Visualization\nWe serve the Professional Visualization market by working closely with independent software vendors, or ISVs, to optimize their offerings for NVIDIA GPUs. Our..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4083,What was the total revenue for NVIDIA in fiscal year 2024?...
1,Unknown Document,N/A,0.4281,"What is the revenue for NVIDIA Corporation for the year ended January 28, 2024?..."
2,Unknown Document,N/A,0.4321,By what percentage did NVIDIA's revenue increase from fiscal year 2023 to 2024?...
3,Unknown Document,N/A,0.4502,"What was the total revenue for NVIDIA Corporation for the year ended January 28, 2024?..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
1,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
2,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
3,Unknown Document,N/A,0.1656,Who is the President and Chief Executive Officer of NVIDIA Corporation?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
1,Document 7323,N/A,ORIGINAL,1.0,"San Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses..."
2,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
3,Document 7274,N/A,ORIGINAL,1.0,"included, among others, testing management’s process for developing the provisions for excess or obsolete inventories and excess product purchasecommitments; evaluating the appropriateness of management’s approach; testing the completeness and accuracy of underlying data used in the approach; andevaluating the reasonableness of management’s assumptions related to future demand and market conditions. Evaluating management’s assumptions related to\nfuture demand and market conditions involved evaluating whether the assumptions used by management were reasonable considering (i) current and pastresults, including historical product life cycle, (ii) the consistency with external market and industry data, and (iii) changes in technology.\n/s/ PricewaterhouseCoopers LLP\nSan Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\n..."


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."


### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [22]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [23]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [29]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="Question_Generation",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Question_Generation-8c96be99' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=84ceee55-a847-486b-a22d-f6782e5b63b3




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4067,What markets does NVIDIA target with its full-stack innovation approach?...
1,Unknown Document,N/A,0.4946,What are the main applications of NVIDIA's computer graphics platforms?...
2,Unknown Document,N/A,0.5127,What industries are mentioned as benefiting from NVIDIA's technology?...
3,Unknown Document,N/A,0.5156,What market does the company serve in relation to NVIDIA GPUs?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 805,N/A,ORIGINAL,1.0,"hardware, a reference sensor set that supports full self-driving capability as well as an open, modular DRIVE software platform for autonomous driving, mapping,and parking services, and intelligent in-vehicle experiences.\nIn addition, we offer a scalable data center-based simulation solution, NVIDIA DRIVE Sim, based on NVIDIA Omniverse software, for digital cockpit\ndevelopment, as well as for testing and validating a self-driving platform. Our unique end-to-end, software-defined approach is designed for continuousinnovation and continuous development, enabling cars to receive over-the-air updates to add new features and capabilities throughout the life of a vehicle.\nBusiness Strategies\nNVIDIA’s key strategies that shape our overall business approach include:\nAdvancing the NVIDIA accelerated computing platform. Our accelerated computing platform can solve complex problems in significantly less time and with\nlower power consumption than alternative computational approaches. Indeed, it ca..."
1,Document 914,N/A,ORIGINAL,1.0,"entertainment and professional visualization applications and create new virtual world and simulation capabilities. Our technologies are instrumental in drivingthe gaming, design, and creative industries forward, as developers leverage our libraries and algorithms to deliver an optimized experience on our GeForce andNVIDIA RTX platforms. Our computer graphics platforms leverage AI end-to-end, from the developer tools and cloud services to the Tensor Cores included in all\nRTX-class GPUs. For example, NVIDIA Avatar Cloud Engine, or ACE, is a suite of technologies that help developers bring digital avatars to life with generativeAI, running in the cloud or locally on the PC. GeForce Experience enhances each gamer’s experience by optimizing their PC’s settings, as well as enabling the\nrecording and sharing of gameplay. Our Studio drivers enhance and accelerate a number of popular creative applications. Omniverse is real-time 3D designcollaboration and virtual world simulation software that..."
2,Document 206,N/A,ORIGINAL,1.0,"Forward-Looking Statements\nThis Annual Report on Form 10-K contains forward-looking statements which are based on our management's beliefs and assumptions and on informationcurrently available to our management. In some cases, you can identify forward-looking statements by terms such as “may,” “will,” “should,” “could,” “goal,”\n“would,” “expect,” “plan,” “anticipate,” “believe,” “estimate,” “project,” “predict,” “potential,” and similar expressions intended to identify forward-lookingstatements. These statements involve known and unknown risks, uncertainties and other factors, which may cause our actual results, performance, time framesor achievements to be materially different from any future results, performance, time frames or achievements expressed or implied by the forward-looking\nstatements. We discuss many of these risks, uncertainties, and other factors in this Annual Report on Form 10-K in greater detail under the heading “RiskFactors.” Given these risks, uncertainties, and ot..."
3,Document 727,N/A,ORIGINAL,1.0,"Professional Visualization\nWe serve the Professional Visualization market by working closely with independent software vendors, or ISVs, to optimize their offerings for NVIDIA GPUs. Our..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4083,What was the total revenue for NVIDIA in fiscal year 2024?...
1,Unknown Document,N/A,0.4281,"What is the revenue for NVIDIA Corporation for the year ended January 28, 2024?..."
2,Unknown Document,N/A,0.4322,By what percentage did NVIDIA's revenue increase from fiscal year 2023 to 2024?...
3,Unknown Document,N/A,0.4502,"What was the total revenue for NVIDIA Corporation for the year ended January 28, 2024?..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
1,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
2,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
3,Unknown Document,N/A,0.1656,Who is the President and Chief Executive Officer of NVIDIA Corporation?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
1,Document 7323,N/A,ORIGINAL,1.0,"San Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses..."
2,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
3,Document 7274,N/A,ORIGINAL,1.0,"included, among others, testing management’s process for developing the provisions for excess or obsolete inventories and excess product purchasecommitments; evaluating the appropriateness of management’s approach; testing the completeness and accuracy of underlying data used in the approach; andevaluating the reasonableness of management’s assumptions related to future demand and market conditions. Evaluating management’s assumptions related to\nfuture demand and market conditions involved evaluating whether the assumptions used by management were reasonable considering (i) current and pastresults, including historical product life cycle, (ii) the consistency with external market and industry data, and (iii) changes in technology.\n/s/ PricewaterhouseCoopers LLP\nSan Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\n..."


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."


### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [26]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [27]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [32]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="Question_Generation",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Question_Generation-abe56717' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=4fbabbcc-a7a5-441d-9747-475cd869c63e




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4083,What was the total revenue for NVIDIA in fiscal year 2024?...
1,Unknown Document,N/A,0.4281,"What is the revenue for NVIDIA Corporation for the year ended January 28, 2024?..."
2,Unknown Document,N/A,0.4322,By what percentage did NVIDIA's revenue increase from fiscal year 2023 to 2024?...
3,Unknown Document,N/A,0.4502,"What was the total revenue for NVIDIA Corporation for the year ended January 28, 2024?..."


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
1,Document 7323,N/A,ORIGINAL,1.0,"San Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses..."
2,Document 5803,N/A,ORIGINAL,1.0,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory pro..."
3,Document 7274,N/A,ORIGINAL,1.0,"included, among others, testing management’s process for developing the provisions for excess or obsolete inventories and excess product purchasecommitments; evaluating the appropriateness of management’s approach; testing the completeness and accuracy of underlying data used in the approach; andevaluating the reasonableness of management’s assumptions related to future demand and market conditions. Evaluating management’s assumptions related to\nfuture demand and market conditions involved evaluating whether the assumptions used by management were reasonable considering (i) current and pastresults, including historical product life cycle, (ii) the consistency with external market and industry data, and (iii) changes in technology.\n/s/ PricewaterhouseCoopers LLP\nSan Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\n..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
1,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
2,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
3,Unknown Document,N/A,0.1656,Who is the President and Chief Executive Officer of NVIDIA Corporation?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4067,What markets does NVIDIA target with its full-stack innovation approach?...
1,Unknown Document,N/A,0.4946,What are the main applications of NVIDIA's computer graphics platforms?...
2,Unknown Document,N/A,0.5127,What industries are mentioned as benefiting from NVIDIA's technology?...
3,Unknown Document,N/A,0.5156,What market does the company serve in relation to NVIDIA GPUs?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 805,N/A,ORIGINAL,1.0,"hardware, a reference sensor set that supports full self-driving capability as well as an open, modular DRIVE software platform for autonomous driving, mapping,and parking services, and intelligent in-vehicle experiences.\nIn addition, we offer a scalable data center-based simulation solution, NVIDIA DRIVE Sim, based on NVIDIA Omniverse software, for digital cockpit\ndevelopment, as well as for testing and validating a self-driving platform. Our unique end-to-end, software-defined approach is designed for continuousinnovation and continuous development, enabling cars to receive over-the-air updates to add new features and capabilities throughout the life of a vehicle.\nBusiness Strategies\nNVIDIA’s key strategies that shape our overall business approach include:\nAdvancing the NVIDIA accelerated computing platform. Our accelerated computing platform can solve complex problems in significantly less time and with\nlower power consumption than alternative computational approaches. Indeed, it ca..."
1,Document 914,N/A,ORIGINAL,1.0,"entertainment and professional visualization applications and create new virtual world and simulation capabilities. Our technologies are instrumental in drivingthe gaming, design, and creative industries forward, as developers leverage our libraries and algorithms to deliver an optimized experience on our GeForce andNVIDIA RTX platforms. Our computer graphics platforms leverage AI end-to-end, from the developer tools and cloud services to the Tensor Cores included in all\nRTX-class GPUs. For example, NVIDIA Avatar Cloud Engine, or ACE, is a suite of technologies that help developers bring digital avatars to life with generativeAI, running in the cloud or locally on the PC. GeForce Experience enhances each gamer’s experience by optimizing their PC’s settings, as well as enabling the\nrecording and sharing of gameplay. Our Studio drivers enhance and accelerate a number of popular creative applications. Omniverse is real-time 3D designcollaboration and virtual world simulation software that..."
2,Document 206,N/A,ORIGINAL,1.0,"Forward-Looking Statements\nThis Annual Report on Form 10-K contains forward-looking statements which are based on our management's beliefs and assumptions and on informationcurrently available to our management. In some cases, you can identify forward-looking statements by terms such as “may,” “will,” “should,” “could,” “goal,”\n“would,” “expect,” “plan,” “anticipate,” “believe,” “estimate,” “project,” “predict,” “potential,” and similar expressions intended to identify forward-lookingstatements. These statements involve known and unknown risks, uncertainties and other factors, which may cause our actual results, performance, time framesor achievements to be materially different from any future results, performance, time frames or achievements expressed or implied by the forward-looking\nstatements. We discuss many of these risks, uncertainties, and other factors in this Annual Report on Form 10-K in greater detail under the heading “RiskFactors.” Given these risks, uncertainties, and ot..."
3,Document 727,N/A,ORIGINAL,1.0,"Professional Visualization\nWe serve the Professional Visualization market by working closely with independent software vendors, or ISVs, to optimize their offerings for NVIDIA GPUs. Our..."


# Complete Evaluation

In [28]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Question_Generation",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Question_Generation-757567cd' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=86c73f0f-42ef-45b5-a6d2-7148e369cc64




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.6729,"Cash provided by operating activities increased in fiscal year 2024 compared to fiscal year 2023, due to growth in revenue. Accounts receivable balance in fiscalyear 2024 reflected $557 million from customer payments received ahead of the invoice due date...."
1,Unknown Document,N/A,0.6835,"What is the amount of cash and cash equivalents reported by NVIDIA as of January 28, 2024?..."
2,Unknown Document,N/A,0.6905,What caused the increase in cash provided by operating activities in fiscal year 2024?...
3,Unknown Document,N/A,0.6906,What was the total amount paid in cash dividends by NVIDIA in fiscal year 2024?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 6475,N/A,ORIGINAL,1.0,"Cash provided by operating activities increased in fiscal year 2024 compared to fiscal year 2023, due to growth in revenue. Accounts receivable balance in fiscalyear 2024 reflected $557 million from customer payments received ahead of the invoice due date...."
1,Document 7274,N/A,ORIGINAL,1.0,"included, among others, testing management’s process for developing the provisions for excess or obsolete inventories and excess product purchasecommitments; evaluating the appropriateness of management’s approach; testing the completeness and accuracy of underlying data used in the approach; andevaluating the reasonableness of management’s assumptions related to future demand and market conditions. Evaluating management’s assumptions related to\nfuture demand and market conditions involved evaluating whether the assumptions used by management were reasonable considering (i) current and pastresults, including historical product life cycle, (ii) the consistency with external market and industry data, and (iii) changes in technology.\n/s/ PricewaterhouseCoopers LLP\nSan Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49\nTable of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\n..."
2,Document 6366,N/A,ORIGINAL,1.0,"Other, net, consists of realized or unrealized gains and losses from investments in non-affiliated entities and the impact of changes in foreign currency rates.Change in Other, net, compared to fiscal year 2023 was driven by changes in value from our non-affiliated investments. Refer to Note 9 of the Notes to theConsolidated Financial Statements in Part IV, Item 15 of this Annual Report on Form 10-K for additional information regarding our investments in non-affiliated\nentities.\nIncome Taxes\nWe recognized income tax expense of $4.1 billion for fiscal year 2024 and income tax benefit of $187 million for fiscal year 2023. Income tax as a percentage of\nincome before income tax was an expense of 12.0% for fiscal year 2024 and a benefit of 4.5% for fiscal year 2023.\nDuring the third quarter of fiscal year 2024, the Internal Revenue Service, or IRS, audit of our federal income tax returns for fiscal years 2018 and 2019 was\nresolved. We recognized a non-cash net benefit of $145 million, relat..."
3,Document 5392,N/A,ORIGINAL,1.0,"legal requirements, and other factors. The program does not obligate NVIDIA to acquire any particular amount of common stock and the program may besuspended at any time at our discretion.\nIn fiscal year 2024, we paid $395 million in quarterly cash dividends. Our cash dividend program and the payment of future cash dividends under that program..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.5485,"What were the total assets for NVIDIA Corporation as of January 28, 2024?..."
1,Unknown Document,N/A,0.6068,"What were the total current assets for NVIDIA Corporation as of January 28, 2024?..."
2,Unknown Document,N/A,0.6122,"How much did NVIDIA Corporation spend on purchases related to property and equipment and intangible assets for the year ended January 29, 2023?..."
3,Unknown Document,N/A,0.6587,"What is the amount of inventories reported by NVIDIA Corporation as of January 28, 2024?..."


...

,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.4067,What markets does NVIDIA target with its full-stack innovation approach?...
1,Unknown Document,N/A,0.4946,What are the main applications of NVIDIA's computer graphics platforms?...
2,Unknown Document,N/A,0.5127,What industries are mentioned as benefiting from NVIDIA's technology?...
3,Unknown Document,N/A,0.5156,What market does the company serve in relation to NVIDIA GPUs?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 805,N/A,ORIGINAL,1.0,"hardware, a reference sensor set that supports full self-driving capability as well as an open, modular DRIVE software platform for autonomous driving, mapping,and parking services, and intelligent in-vehicle experiences.\nIn addition, we offer a scalable data center-based simulation solution, NVIDIA DRIVE Sim, based on NVIDIA Omniverse software, for digital cockpit\ndevelopment, as well as for testing and validating a self-driving platform. Our unique end-to-end, software-defined approach is designed for continuousinnovation and continuous development, enabling cars to receive over-the-air updates to add new features and capabilities throughout the life of a vehicle.\nBusiness Strategies\nNVIDIA’s key strategies that shape our overall business approach include:\nAdvancing the NVIDIA accelerated computing platform. Our accelerated computing platform can solve complex problems in significantly less time and with\nlower power consumption than alternative computational approaches. Indeed, it ca..."
1,Document 914,N/A,ORIGINAL,1.0,"entertainment and professional visualization applications and create new virtual world and simulation capabilities. Our technologies are instrumental in drivingthe gaming, design, and creative industries forward, as developers leverage our libraries and algorithms to deliver an optimized experience on our GeForce andNVIDIA RTX platforms. Our computer graphics platforms leverage AI end-to-end, from the developer tools and cloud services to the Tensor Cores included in all\nRTX-class GPUs. For example, NVIDIA Avatar Cloud Engine, or ACE, is a suite of technologies that help developers bring digital avatars to life with generativeAI, running in the cloud or locally on the PC. GeForce Experience enhances each gamer’s experience by optimizing their PC’s settings, as well as enabling the\nrecording and sharing of gameplay. Our Studio drivers enhance and accelerate a number of popular creative applications. Omniverse is real-time 3D designcollaboration and virtual world simulation software that..."
2,Document 206,N/A,ORIGINAL,1.0,"Forward-Looking Statements\nThis Annual Report on Form 10-K contains forward-looking statements which are based on our management's beliefs and assumptions and on informationcurrently available to our management. In some cases, you can identify forward-looking statements by terms such as “may,” “will,” “should,” “could,” “goal,”\n“would,” “expect,” “plan,” “anticipate,” “believe,” “estimate,” “project,” “predict,” “potential,” and similar expressions intended to identify forward-lookingstatements. These statements involve known and unknown risks, uncertainties and other factors, which may cause our actual results, performance, time framesor achievements to be materially different from any future results, performance, time frames or achievements expressed or implied by the forward-looking\nstatements. We discuss many of these risks, uncertainties, and other factors in this Annual Report on Form 10-K in greater detail under the heading “RiskFactors.” Given these risks, uncertainties, and ot..."
3,Document 727,N/A,ORIGINAL,1.0,"Professional Visualization\nWe serve the Professional Visualization market by working closely with independent software vendors, or ISVs, to optimize their offerings for NVIDIA GPUs. Our..."


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
1,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
2,Unknown Document,N/A,0.1655,Who is the President and Chief Executive Officer of NVIDIA Corporation?...
3,Unknown Document,N/A,0.1656,Who is the President and Chief Executive Officer of NVIDIA Corporation?...


,Document Name,Page Number,Document Type,Similarity Score,Retrieved Chunk
0,Document 10095,N/A,ORIGINAL,1.0,"* Filed herewith.\n+ Management contract or compensatory plan or arrangement.\n# In accordance with Item 601(b)(32)(ii) of Regulation S-K and SEC Release Nos. 33-8238 and 34-47986, Final Rule: Management's Reports on Internal Control Over Financial Reporting and Certification of\nDisclosure in Exchange Act Periodic Reports, the certifications furnished in Exhibits 32.1 and 32.2 hereto are deemed to accompany this Annual Report on Form 10-K and will not be deemed “filed” forpurpose of Section 18 of the Exchange Act. Such certifications will not be deemed to be incorporated by reference into any filing under the Securities Act or the Exchange Act, except to the extent that theregistrant specifically incorporates it by reference.\n^ Certain exhibits and schedules have been omitted in accordance with Regulation S-K Item 601(a)(5).\nCopies of above exhibits not contained herein are available to any shareholder upon written request to:Investor Relations: NVIDIA Corporation, 2788 San Tomas Expre..."
1,Document 10150,N/A,ORIGINAL,1.0,"Not Applicable.\n83\nSignatures\nPursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalfby the undersigned, thereunto duly authorized, on February 21, 2024.\nNVIDIA Corporation\nBy: /s/ Jen-Hsun Huang \n Jen-Hsun Huang\n President and Chief Executive Officer\nPower of Attorney..."
2,Document 10319,N/A,ORIGINAL,1.0,"(a) all significant deficiencies and material weaknesses in the design or operation of internal control over financial reporting which are reasonably\nlikely to adversely affect the registrant's ability to record, process, summarize and report financial information; and\n(b) any fraud, whether or not material, that involves management or other employees who have a significant role in the registrant's internal\ncontrol over financial reporting.\nDate: February 21, 2024\n \n \n/s/ JEN-HSUN HUANG\nJen-Hsun Huang \nPresident and Chief Executive Officer\nEXHIBIT 31.2\n \nCERTIFICATION\n \nI, Colette M. Kress, certify that:\n1. I have reviewed this Annual Report on Form 10-K of NVIDIA Corporation;\n \n2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to\nmake the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period\ncovered by this report;\n ..."
3,Document 10422,N/A,ORIGINAL,1.0,"Executive Vice President and Chief Financial Officer\nEXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and\n2. The information contained in the Periodic Report fairly presents, in all material respects, the financial condition of the Company at the end of\nthe period covered by the Periodic Report and results of operations of the Company for the period covered by the Periodic Report.\nDate: Fe..."
